In this notebook different autoencoder architectures are implemented: Each architecture is based on an encoding-module and a prediction module. The window size refers to how many timesteps are used to make a prediction. "Multivariate" models process input data of multiple areas at the same time to predict demand. 

# Import Modules

In [2]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime

import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import itertools

import os

#from tqdm import tqdm

from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.layers import Concatenate
from keras.layers import Dropout

from keras import optimizers
from keras.models import model_from_json
from keras import regularizers

from keras.callbacks import EarlyStopping

import save_files_collection as sf

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [6]:
#load data
TRAIN_PATH = '/media/...'
Store_PATH = '/media/...'
file_name = 'ts_10largest.csv'
ts_10largest = pd.read_csv(TRAIN_PATH + file_name, header=0, parse_dates=['date'], index_col = 'date')

# Data Preparation

In [8]:
''' Time Series Preparation '''

#Preprocessing Steps: 1) Differencing 2) Scaling 3) Supervised_Sequence

#make timeseries stationary
def differencing(dataset,shift_interval):
    shifted = dataset.shift(shift_interval).copy(deep=True)
    diff_series = dataset - shifted
    diff_series.dropna(inplace=True)
    return diff_series



def invert_differencing(history, predictions, year_to_access):
    history_copy = history.copy()
    history_shift = history_copy.shift(1)
    history_shift.dropna(inplace=True)
    preds_all = predictions + history_shift.loc[year_to_access]
    
    return preds_all
    
    

In [9]:

#it is recommended to scale input data in the range of the activation function used by the network. Since we use "tanh"..
# scale datasets based on fit of train_set; input set has to be a reshaped numpy_array!
def data_scaler(train_data, valid_data, test_data, scale_range, standardizing_flag=False):
        
    if standardizing_flag == False:  
        print('MinMax-Scaling used...')
        #use MinMax-Scaling based on scale_range given...
        scaler = MinMaxScaler(feature_range=scale_range) #feature_range=(-1, 1)
        scaler = scaler.fit(train_data)
        #scale train_set:
        train_scaled = scaler.transform(train_data)
        train_scaled_df = pd.DataFrame(train_scaled)
        #scale valididation data based on scaler fitted on training data:
        valid_scaled = scaler.transform(valid_data)
        valid_scaled_df = pd.DataFrame(valid_scaled)
        #scale test_set based on scaler fitted on training data:
        test_scaled = scaler.transform(test_data)
        test_scaled_df = pd.DataFrame(test_scaled)
            
    else:  
        print('Standardizing used...')
        
        scaler = StandardScaler()
        scaler = scaler.fit(train_data)
        #scale train_set:
        train_scaled = scaler.transform(train_data)
        train_scaled_df = pd.DataFrame(train_scaled)
        #scale valididation data based on scaler fitted on training data:
        valid_scaled = scaler.transform(valid_data)
        valid_scaled_df = pd.DataFrame(valid_scaled)
        #scale test_set based on scaler fitted on training data:
        test_scaled = scaler.transform(test_data)
        test_scaled_df = pd.DataFrame(test_scaled)
        
        
    #return data as df, since "supervised" function requires df or series as input
    return scaler, train_scaled_df, valid_scaled_df, test_scaled_df


#function to invert scaling:
def invert_data_scaler(scaler, predictions_data, standardizing_flag, scale_range, n_preds):
    
    if standardizing_flag == False: 
        #create new scaler: -> necessary since old scaler might be fitted on lagged values -> expects more dimensions as the predictions have! 
        new_scaler = MinMaxScaler(feature_range=scale_range)
        #copy attributes of old scaler to new one: but only for the first columns for which we have real predictions
        new_scaler.min_, new_scaler.scale_ = scaler.min_[:n_preds], scaler.scale_[:n_preds]

        preds = new_scaler.inverse_transform(predictions_data)
        
    else:
        new_scaler = StandardScaler()
        #copy attributes of old scaler to new one: but only for the first columns for which we have real predictions
        new_scaler.mean_, new_scaler.scale_ = scaler.mean_[:n_preds], scaler.scale_[:n_preds]

        preds = new_scaler.inverse_transform(predictions_data)

    
    return preds




In [11]:
def create_supervised_data_single_ts(single_ts, n_timesteps):
    #copy dfs to ensure no changes are made on original dataset:
    sequence_copy = single_ts.copy(deep=True)
    #create dfs to easily append new columns and access columns:
    sequence_copy = pd.DataFrame(sequence_copy)
    sequence_df = pd.DataFrame(sequence_copy)
    
    #Note: range starts reversed to make sure the lags are in correct order
    for i in range(n_timesteps,0,-1): 
        sequence_df['lag_{}'.format(i)] = sequence_copy.iloc[:,0].shift(i)
    
       
    #drop rows with NaNs -> if no lagged features are available, we drop the whole row
    sequence_df.dropna(inplace=True) 

    return sequence_df  


In [12]:
def create_supervised_UBER_data_single_ts(single_ts, n_timesteps_T, n_timesteps_F):
    #copy dfs to ensure no changes are made on original dataset:
    sequence_copy = single_ts.copy(deep=True)
    #create dfs to easily append new columns and access columns:
    sequence_copy = pd.DataFrame(sequence_copy)
    sequence_df = pd.DataFrame(sequence_copy)
    
    #Note: range starts reversed to make sure the lags are in correct order
    for i in range(n_timesteps_T,0,-1): 
        sequence_df['lag_T{}'.format(i)] = sequence_copy.iloc[:,0].shift(i)
    for i in range(n_timesteps_F,0,-1): 
        sequence_df['prev_lag_F{}'.format(i)] = sequence_copy.iloc[:,0].shift(i)
    #Note: for future values range starts "normal":
    for i in range(1,n_timesteps_F+1): 
        sequence_df['fut_{}'.format(i)] = sequence_copy.iloc[:,0].shift(-i)

       
    #drop rows with NaNs -> if no lagged features are available, we drop the whole row
    sequence_df.dropna(inplace=True) 

    return sequence_df  


# preprocess data for models:

In [13]:
def generate_UBER_data_autoencoder(ts_series, multivariate_flag, last_train_set_year, 
                                   validation_set_year, test_set_year, n_timesteps_T, n_timesteps_F, n_preds, 
                                   scale_range, standardizing_flag):         
    #prepare data:
    
    # 1) apply differencing
    ts_diff = differencing(ts_series,1)

    #reassign df:
    ts_series = ts_diff
    
    # 2) get supervised data:
    if multivariate_flag == False:
        #change type of ts_series:
        ts_series = pd.DataFrame(ts_series) #-> this way we can access the "column" parameter
    
    #prepare dict to store results of each area:
    area_labels = list(ts_series.columns)
    area_supervised_dict = {}
    scaler_list = []
    
    #get data for each area:
    for i in range(len(area_labels)):
        print('data for area{} is prepared...'.format(area_labels[i]))
        #create key:
        area_supervised_dict['area{}'.format(area_labels[i])] = []
        #get supervised data:
        supervised_data_df = create_supervised_UBER_data_single_ts(ts_series.iloc[:,i], n_timesteps_T, n_timesteps_F)
        
        
        #get train/test split:      
        ts_train_all, ts_test = supervised_data_df.loc[:validation_set_year], supervised_data_df.loc[test_set_year]             
        #get train/validation Split:
        ts_train, ts_valid = ts_train_all.loc[:last_train_set_year], ts_train_all.loc[validation_set_year]
                
           
        #scale data:
        #create numpy arrays for scaler:
        ts_train_array = ts_train.values
        ts_valid_array = ts_valid.values
        ts_test_array = ts_test.values
        
        print('Data is scaled...')
        scaler, train_scaled, valid_scaled, test_scaled = data_scaler(ts_train_array, ts_valid_array, ts_test_array, scale_range, standardizing_flag)
        
        #store scaler:
        scaler_list.append(scaler)
        
        #restore index:
        train_scaled.index = ts_train.index
        valid_scaled.index = ts_valid.index
        test_scaled.index = ts_test.index

        
        #slice lags_T, lags_F, future_F values:
        #slice lags_T which are actually X:
        X_train = train_scaled.iloc[:,1:(n_timesteps_T+1)].values
        X_valid = valid_scaled.iloc[:,1:(n_timesteps_T+1)].values
        X_test = test_scaled.iloc[:,1:(n_timesteps_T+1)].values
        #get y values:
        y_train = train_scaled.iloc[:,0].values
        y_valid = valid_scaled.iloc[:,0].values
        y_test = test_scaled.iloc[:,0].values  
        #slice lags_F:
        lags_F_train = train_scaled.iloc[:,-(n_timesteps_F + n_timesteps_F):-(n_timesteps_F)].values
        lags_F_valid = valid_scaled.iloc[:,-(n_timesteps_F + n_timesteps_F):-(n_timesteps_F)].values
        lags_F_test = test_scaled.iloc[:,-(n_timesteps_F + n_timesteps_F):-(n_timesteps_F)].values
        #slice future_F:
        fut_F_train = train_scaled.iloc[:,-(n_timesteps_F):].values
        fut_F_valid = valid_scaled.iloc[:,-(n_timesteps_F):].values
        fut_F_test = test_scaled.iloc[:,-(n_timesteps_F):].values
        
        
        #reshape X, y and lags_F and fut_F data to easily append other areas later on:
        list_to_reshape_X_lags_F_fut_F = [X_train, X_valid, X_test, lags_F_train, lags_F_valid, lags_F_test, fut_F_train, fut_F_valid, fut_F_test] 
        list_to_reshape_y = [y_train, y_valid, y_test]
        reshaped_list_X_lags_F_fut_F = []
        reshaped_list_y = []
        #reshape X:
        for array in list_to_reshape_X_lags_F_fut_F:
            #shape: (#n_samples,n_lags,n_areas)
            array = array.reshape((array.shape[0],array.shape[1],1))
            reshaped_list_X_lags_F_fut_F.append(array)
            
        #reshape y:
        for array in list_to_reshape_y:
            #shape: (#n_samples,n_areas)
            array = array.reshape((array.shape[0],1))
            reshaped_list_y.append(array)
            
           
        #append results to dict:
        items_to_append = [reshaped_list_X_lags_F_fut_F[0], reshaped_list_y[0], reshaped_list_X_lags_F_fut_F[3], reshaped_list_X_lags_F_fut_F[6],
                          reshaped_list_X_lags_F_fut_F[1], reshaped_list_y[1], reshaped_list_X_lags_F_fut_F[4], reshaped_list_X_lags_F_fut_F[7],
                          reshaped_list_X_lags_F_fut_F[2], reshaped_list_y[2], reshaped_list_X_lags_F_fut_F[5], reshaped_list_X_lags_F_fut_F[8]]
        
        for u in range(len(items_to_append)):
            area_supervised_dict['area{}'.format(area_labels[i])].append(items_to_append[u])
    
        
        #print shapes:
        print('shape of X_train single area: ', reshaped_list_X_lags_F_fut_F[0].shape)
        print('shape of X_valid single area: ', reshaped_list_X_lags_F_fut_F[1].shape)
        print('shape of X_test single area: ', reshaped_list_X_lags_F_fut_F[2].shape)
        print('shape of y_train single area: ', reshaped_list_y[0].shape)
        print('shape of y_valid single area: ', reshaped_list_y[1].shape)
        print('shape of y_test single area: ', reshaped_list_y[2].shape)
        print('shape of prev_lags_F_train single area: ', reshaped_list_X_lags_F_fut_F[3].shape)
        print('shape of fut_F_values_train single area: ', reshaped_list_X_lags_F_fut_F[6].shape)
    
    
    
    #concat results of all areas:
    key_list = list(area_supervised_dict.keys())
    
    if multivariate_flag == True:
        #create training set, valid & test set containing inputs of all selected areas: -> append all areas into one big np.array!

        #fill arrays with entries of first area:
        X_train, y_train = area_supervised_dict[key_list[0]][0], area_supervised_dict[key_list[0]][1]
        X_valid, y_valid = area_supervised_dict[key_list[0]][4], area_supervised_dict[key_list[0]][5]
        X_test, y_test = area_supervised_dict[key_list[0]][8], area_supervised_dict[key_list[0]][9]
        
        lags_F_train = area_supervised_dict[key_list[0]][2]
        lags_F_valid = area_supervised_dict[key_list[0]][6]
        lags_F_test = area_supervised_dict[key_list[0]][10]

        fut_F_train = area_supervised_dict[key_list[0]][3]
        fut_F_valid = area_supervised_dict[key_list[0]][7]
        fut_F_test = area_supervised_dict[key_list[0]][11]
 
        
        #concat remaining areas:
        for i in range(1,len(key_list)):
            X_train = np.concatenate((X_train, area_supervised_dict[key_list[i]][0]),axis=2)
            X_valid = np.concatenate((X_valid, area_supervised_dict[key_list[i]][4]),axis=2)
            X_test = np.concatenate((X_test, area_supervised_dict[key_list[i]][8]),axis=2)
            y_train = np.concatenate((y_train, area_supervised_dict[key_list[i]][1]),axis=1)
            y_valid = np.concatenate((y_valid, area_supervised_dict[key_list[i]][5]),axis=1)
            y_test = np.concatenate((y_test, area_supervised_dict[key_list[i]][9]),axis=1)
            
            lags_F_train = np.concatenate((lags_F_train, area_supervised_dict[key_list[i]][2]),axis=2)
            lags_F_valid = np.concatenate((lags_F_valid, area_supervised_dict[key_list[i]][6]),axis=2)
            lags_F_test = np.concatenate((lags_F_test, area_supervised_dict[key_list[i]][10]),axis=2)

            fut_F_train = np.concatenate((fut_F_train, area_supervised_dict[key_list[i]][3]),axis=2)
            fut_F_valid = np.concatenate((fut_F_valid, area_supervised_dict[key_list[i]][7]),axis=2)
            fut_F_test = np.concatenate((fut_F_test, area_supervised_dict[key_list[i]][11]),axis=2)
     
    
    else:
        X_train, y_train = area_supervised_dict[key_list[0]][0], area_supervised_dict[key_list[0]][1]
        X_valid, y_valid = area_supervised_dict[key_list[0]][4], area_supervised_dict[key_list[0]][5]
        X_test, y_test = area_supervised_dict[key_list[0]][8], area_supervised_dict[key_list[0]][9]
        
        lags_F_train = area_supervised_dict[key_list[0]][2]
        lags_F_valid = area_supervised_dict[key_list[0]][6]
        lags_F_test = area_supervised_dict[key_list[0]][10]

        fut_F_train = area_supervised_dict[key_list[0]][3]
        fut_F_valid = area_supervised_dict[key_list[0]][7]
        fut_F_test = area_supervised_dict[key_list[0]][11]
        
    
    print('final shape of X_train: ', X_train.shape)
    
    #return area_supervised_dict
    
    
    #Note: We only need to return lags_F & fut_F_values for train and valid set since autoencoder is only "trained" on these sets..
    return (X_train, y_train, X_valid, y_valid, X_test, y_test, lags_F_train, lags_F_valid, fut_F_train, 
            fut_F_valid, scaler_list)


    

    

In [14]:
def generate_data_autoencoder(ts_series, multivariate_flag, last_train_set_year, validation_set_year, test_set_year, 
                              n_timesteps, n_preds, scale_range, standardizing_flag):
    #prepare data:
    
    # 1) apply differencing
    ts_diff = differencing(ts_series,1)

    #reassign df:
    ts_series = ts_diff
    
        # 2) get supervised data:
    if multivariate_flag == False:
        #change type of ts_series:
        ts_series = pd.DataFrame(ts_series) #-> this way we can access the "column" parameter
    
    #prepare dict to store results of each area:
    area_labels = list(ts_series.columns)
    area_supervised_dict = {}
    scaler_list = []
    
    #get data for each area:
    for i in range(len(area_labels)):
        print('data for area{} is prepared...'.format(area_labels[i]))
        #create key:
        area_supervised_dict['area{}'.format(area_labels[i])] = []
        #get supervised data:
        supervised_data_df = create_supervised_data_single_ts(ts_series.iloc[:,i], n_timesteps)
        
        
        #get train/test split:      
        ts_train_all, ts_test = supervised_data_df.loc[:validation_set_year], supervised_data_df.loc[test_set_year]             
        #get train/validation Split:
        ts_train, ts_valid = ts_train_all.loc[:last_train_set_year], ts_train_all.loc[validation_set_year]

           
        #scale data:
        #create numpy arrays for scaler:
        ts_train_array = ts_train.values
        ts_valid_array = ts_valid.values
        ts_test_array = ts_test.values
        
        print('Data is scaled...')
        scaler, train_scaled, valid_scaled, test_scaled = data_scaler(ts_train_array, ts_valid_array, ts_test_array, scale_range, standardizing_flag)
        
        #store scaler:
        scaler_list.append(scaler)
        
        #restore index:
        train_scaled.index = ts_train.index
        valid_scaled.index = ts_valid.index
        test_scaled.index = ts_test.index

        
        #get X, y pairs:
        X_train = train_scaled.iloc[:,1:(n_timesteps_T+1)].values
        X_valid = valid_scaled.iloc[:,1:(n_timesteps_T+1)].values
        X_test = test_scaled.iloc[:,1:(n_timesteps_T+1)].values
        #get y values:
        y_train = train_scaled.iloc[:,0].values
        y_valid = valid_scaled.iloc[:,0].values
        y_test = test_scaled.iloc[:,0].values  
      
        
        #reshape X, y to easily append other areas later on:
        list_to_reshape_X = [X_train, X_valid, X_test] 
        list_to_reshape_y = [y_train, y_valid, y_test]
        reshaped_list_X = []
        reshaped_list_y = []
        #reshape X:
        for array in list_to_reshape_X:
            #shape: (#n_samples,n_lags,n_areas)
            array = array.reshape((array.shape[0],array.shape[1],1))
            reshaped_list_X.append(array)
            
        #reshape y:
        for array in list_to_reshape_y:
            #shape: (#n_samples,n_areas)
            array = array.reshape((array.shape[0],1))
            reshaped_list_y.append(array)
            
           
        #append results to dict:
        items_to_append = [reshaped_list_X[0], reshaped_list_y[0], 
                          reshaped_list_X[1], reshaped_list_y[1], 
                          reshaped_list_X[2], reshaped_list_y[2]]
        
        for u in range(len(items_to_append)):
            area_supervised_dict['area{}'.format(area_labels[i])].append(items_to_append[u])
    
        
        #print shapes:
        print('shape of X_train single area: ', reshaped_list_X[0].shape)
        print('shape of X_valid single area: ', reshaped_list_X[1].shape)
        print('shape of X_test single area: ', reshaped_list_X[2].shape)
        print('shape of y_train single area: ', reshaped_list_y[0].shape)
        print('shape of y_valid single area: ', reshaped_list_y[1].shape)
        print('shape of y_test single area: ', reshaped_list_y[2].shape)    
    
    
    #concat results of all areas:
    key_list = list(area_supervised_dict.keys())
    
    if multivariate_flag == True:
        #create training set, valid & test set containing inputs of all selected areas: -> append all areas into one big np.array!

        #fill arrays with entries of first area:
        X_train, y_train = area_supervised_dict[key_list[0]][0], area_supervised_dict[key_list[0]][1]
        X_valid, y_valid = area_supervised_dict[key_list[0]][2], area_supervised_dict[key_list[0]][3]
        X_test, y_test = area_supervised_dict[key_list[0]][4], area_supervised_dict[key_list[0]][5]
                
        #concat remaining areas:
        for i in range(1,len(key_list)):
            X_train = np.concatenate((X_train, area_supervised_dict[key_list[i]][0]),axis=2)
            X_valid = np.concatenate((X_valid, area_supervised_dict[key_list[i]][2]),axis=2)
            X_test = np.concatenate((X_test, area_supervised_dict[key_list[i]][4]),axis=2)
            y_train = np.concatenate((y_train, area_supervised_dict[key_list[i]][1]),axis=1)
            y_valid = np.concatenate((y_valid, area_supervised_dict[key_list[i]][3]),axis=1)
            y_test = np.concatenate((y_test, area_supervised_dict[key_list[i]][5]),axis=1)
                 
    
    else:
        X_train, y_train = area_supervised_dict[key_list[0]][0], area_supervised_dict[key_list[0]][1]
        X_valid, y_valid = area_supervised_dict[key_list[0]][2], area_supervised_dict[key_list[0]][3]
        X_test, y_test = area_supervised_dict[key_list[0]][4], area_supervised_dict[key_list[0]][5]
        
    print('final shape of X_train: ', X_train.shape)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test, scaler_list
    
    

# Model Definition

## simpler autoencoder with either 1 hidden layer or 2 hidden layers

In [15]:
def create_autoencoder_1H(X_train, X_valid, n_timesteps, n_preds, n_hidden1_units, dropout_rate, n_batch_size_training, n_epochs):
    #create encoder with Keras Functional API:
    # define encoder
    input1 = Input(shape=(X_train.shape[1], X_train.shape[2]))  
    encoder = LSTM(n_hidden1_units, activation='tanh', dropout=dropout_rate)(input1) 
    #encoder layer returns a static meaningful vector -> to receive a sequence again, we use RepeatVector which returns the meaningful vector "n" times
    # define reconstruct decoder
    encoder2 = RepeatVector(n_timesteps)(encoder) #needed to keep the 3D shape (X_train.shape[1])
    decoder = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(encoder2) #previously 32units
    decoder = TimeDistributed(Dense(n_preds))(decoder)

    autoencoder_model_ts = Model(inputs=input1, outputs=decoder)
    autoencoder_model_ts.compile(optimizer='adam', loss='mse')
    # fit model
    history = autoencoder_model_ts.fit(X_train, X_train, batch_size=n_batch_size_training, epochs=n_epochs, validation_data=(X_valid, X_valid), verbose=1, shuffle=True)
    return history, autoencoder_model_ts


In [16]:
def create_stacked_autoencoder_2H(X_train, X_valid, n_timesteps, n_preds, n_hidden1_units, n_hidden2_units, dropout_rate, n_batch_size_training, n_epochs):
    #create encoder with Keras Functional API:
    # define encoder
    input1 = Input(shape=(X_train.shape[1], X_train.shape[2]))  
    encoder1 = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(input1)
    encoder2 = LSTM(n_hidden2_units, activation='tanh', dropout=dropout_rate)(encoder1)
    #encoder2 layer returns a static meaningful vector -> to receive a sequence again, we use RepeatVector..
    #define reconstruct decoder
    encoder3 = RepeatVector(n_timesteps)(encoder2) #RepeatVector actually belongs to the encoding-module -> name = "encoder3"
    decoder1 = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(encoder3)
    decoder2 = TimeDistributed(Dense(n_preds))(decoder1)

    autoencoder_model_ts = Model(inputs=input1, outputs=decoder2)
    autoencoder_model_ts.compile(optimizer='adam', loss='mse')
    # fit model
    history = autoencoder_model_ts.fit(X_train, X_train, batch_size=n_batch_size_training, epochs=n_epochs, validation_data=(X_valid, X_valid), verbose=1, shuffle=True)
    return history, autoencoder_model_ts


In [17]:
def create_prediction_model_2H(X_train_encoded, y_train, X_valid_encoded, y_valid, n_preds, n_hidden1_units, n_hidden2_units, dropout_rate, n_batch_size_training, n_epochs, shuffle_flag, early_stopping_flag):     
    
    #create callback list for different callbacks:
    callback_list = []
    
    if early_stopping_flag == True:
        print('Early Stopping applied')
        #create Callback for EarlyStopping:
        early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
        #append callback to list:
        callback_list.append(early_stop)

    
    # define prediction model:
    input2 = Input(shape=(X_train_encoded.shape[1], X_train_encoded.shape[2]))
    lstm_2 = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(input2)
    lstm_2 = LSTM(n_hidden2_units, activation='tanh', return_sequences=False, dropout=dropout_rate)(lstm_2)
    #dense2 = Dense(50)(lstm_2)
    out2 = Dense(n_preds)(lstm_2)

    predict_model = Model(input2, out2)
    predict_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    history = predict_model.fit(X_train_encoded, y_train, epochs=n_epochs, batch_size=n_batch_size_training, validation_data=(X_valid_encoded, y_valid), verbose=1, shuffle=shuffle_flag, callbacks = callback_list)
    return history, predict_model


## UBER autoencoder with either 1 hidden layer or 2 hidden layers

In [19]:
'''Note: There are different ways how the autoencoder can be constructed:
    1) LSTM LAyers of Encoding Scheme don't use return_sequences -> this way a static embedding is returned after all timesteps are processed, -> a RepeatVector-Layer is needed which creates multiple copies of the static embedding
    2) LSTM LAyers of Encoding Scheme use return_sequences -> The hidden state of each time step is returned -> we don't need RepeatVector-Layer
    
    --> here: repeatVector used in this Version
'''
def create_UBER_autoencoder_1H_with_RepeatVect(X_train_T, X_train_F_prev, X_train_F_fut, X_valid_T, X_valid_F_prev, X_valid_F_fut, n_timesteps_F, n_preds, n_hidden1_units, n_hidden2_units, dropout_rate, n_batch_size_training, n_epochs):
    #create encoder with Keras Functional API:
    # define encoder
    #define input of regular sliding window sequence (previous 't' days)
    input1 = Input(shape=(X_train_T.shape[1], X_train_T.shape[2]))
    #create LSTM layers:
    encoder1 = LSTM(n_hidden1_units, activation='tanh', return_sequences=False, dropout=dropout_rate)(input1)
    #encoder1 layer returns a static meaningful vector -> to receive a sequence again, we use RepeatVector..
    #define reconstruct decoder
    encoder2 = RepeatVector(n_timesteps_F)(encoder1) #RepeatVector actually belongs to the encoding-module -> name = "encoder3"
    #define input of additional sliding window sequence (previous 'F' days of target) -> defined in UBER Paper.. 
    input2 = Input(shape=(X_train_F_prev.shape[1], X_train_F_prev.shape[2]))
    #create concat layer to add additional sequence input on axis=2:
    concat1 = Concatenate(axis=2)([encoder2,input2])
    #Decoding Module:
    decoder1 = LSTM(n_hidden2_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(concat1) #decoder1 returns sequence of hidden states
    decoder2 = TimeDistributed(Dense(n_preds))(decoder1) #Note: "TimeDistributed" applies Dense-Layer on each sequence returned by decoder1 

    autoencoder_model_ts = Model(inputs=[input1,input2], outputs=decoder2)
    autoencoder_model_ts.compile(optimizer='adam', loss='mse')
    # fit model
    history = autoencoder_model_ts.fit([X_train_T, X_train_F_prev], X_train_F_fut, batch_size=n_batch_size_training, epochs=n_epochs, validation_data=([X_valid_T, X_valid_F_prev], X_valid_F_fut), verbose=1, shuffle=True)
    
    return history, autoencoder_model_ts



In [20]:
''' --> here: repeatVector used in this Version '''
def create_stacked_UBER_autoencoder_2H(X_train_T, X_train_F_prev, X_train_F_fut, X_valid_T, X_valid_F_prev, X_valid_F_fut, n_timesteps_F, n_preds, n_hidden1_units, n_hidden2_units, dropout_rate, n_batch_size_training, n_epochs):
    #create encoder with Keras Functional API:
    # define encoder
    #define input of regular sliding window sequence (previous 't' days)
    input1 = Input(shape=(X_train_T.shape[1], X_train_T.shape[2]))
    #create LSTM layers:
    encoder1 = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(input1)
    encoder2 = LSTM(n_hidden2_units, activation='tanh', dropout=dropout_rate)(encoder1)
    #encoder2 layer returns a static meaningful vector -> to receive a sequence again, we use RepeatVector..
    #define reconstruct decoder
    encoder3 = RepeatVector(n_timesteps_F)(encoder2) #RepeatVector actually belongs to the encoding-module -> name = "encoder3"
    #define input of additional sliding window sequence (previous 'F' days of target) -> defined in UBER Paper.. 
    input2 = Input(shape=(X_train_F_prev.shape[1], X_train_F_prev.shape[2]))
    #create concat layer to add additional sequence input on axis=2:
    concat1 = Concatenate(axis=2)([encoder3,input2])
    #Decoding Module:
    decoder1 = LSTM(n_hidden1_units, activation='tanh', return_sequences=True, dropout=dropout_rate)(concat1)
    decoder2 = TimeDistributed(Dense(n_preds))(decoder1)

    autoencoder_model_ts = Model(inputs=[input1,input2], outputs=decoder2)
    autoencoder_model_ts.compile(optimizer='adam', loss='mse')
    # fit model
    history = autoencoder_model_ts.fit([X_train_T, X_train_F_prev], X_train_F_fut, batch_size=n_batch_size_training, epochs=n_epochs, validation_data=([X_valid_T, X_valid_F_prev], X_valid_F_fut), verbose=1, shuffle=True)
    
    return history, autoencoder_model_ts



In [21]:
def create_UBER_prediction_model_3H(X_train_encoded, y_train, X_valid_encoded, y_valid, n_preds, n_hidden1_units, n_hidden2_units, n_hidden3_units, dropout_rate, n_batch_size_training, n_epochs, shuffle_flag):     
    # define prediction model:
    #we only need 2D shape for Dense-Layers..
    input_data = Input(shape=(X_train_encoded.shape[1], ))
    dense1 = Dense(n_hidden1_units, activation='tanh')(input_data)
    dropout1 = Dropout(dropout_rate)(dense1)
    dense2 = Dense(n_hidden2_units, activation='tanh')(dropout1)
    dropout2 = Dropout(dropout_rate)(dense2)
    dense3 = Dense(n_hidden3_units, activation='tanh')(dropout2)
    dropout3 = Dropout(dropout_rate)(dense3)
    out2 = Dense(n_preds)(dropout3)
    
    #compile model:
    predict_model = Model(input_data, out2)
    predict_model.compile(loss='mse', optimizer='adam', metrics=['mae'])

    history = predict_model.fit(X_train_encoded, y_train, epochs=n_epochs, batch_size=n_batch_size_training, validation_data=(X_valid_encoded, y_valid), verbose=1, shuffle=shuffle_flag)
    
    return history, predict_model


# get predictions 

In [22]:
#get prediction for scaled + differenced input_data (assuming scaled input) !!!!!!:

#assuming encoding & reshaping for input_data was already done 

def get_rescaled_decoded_predictions(model, multivariate_flag, X_test_encoded, year_to_access, scaler_list, standardizing_flag, scale_range, n_preds, original_complete_dataset, model_name):       
    
    # 1) get predictions:
    print('get predictions of model...')
    yhat = model.predict(X_test_encoded, verbose = 1)
    #print(yhat.shape)
    #print(yhat)
    
    print('yhat shape: ', yhat.shape)
    
    # 2) rescale predictions (use scaler of each area):
    if multivariate_flag == False:
        yhat_rescaled_all = invert_data_scaler(scaler_list[0], yhat, standardizing_flag, scale_range, n_preds)
    
    else:
        yhat_rescaled_all_list = []
        for i in range(len(scaler_list)):
            #slice predictions for each area and rescale predictions: (Note: slicing columns of numpy array returns a list! -> reshaping necessary afterwards)
            yhat_area_i = yhat[:,i]
            #reshape unscaled predictions for scaler:
            yhat_area_i = yhat_area_i.reshape(len(yhat_area_i),1)
            #apply scaler:
            yhat_rescaled_area_i = invert_data_scaler(scaler_list[i], yhat_area_i, standardizing_flag, scale_range, 1) #Note: for multivariate case n_preds is set to "1" since we only take first column of scaler for each area
            yhat_rescaled_all_list.append(yhat_rescaled_area_i)
    
        #restore numpy_array based on yhat_rescaled_all_list: (this way we have all rescaled predictions for each area in one big numpy array)
        yhat_rescaled_all = yhat_rescaled_all_list[0]
        for i in range(1,len(yhat_rescaled_all_list)):
            yhat_rescaled_all = np.concatenate((yhat_rescaled_all,yhat_rescaled_all_list[i]),axis=1)
        
    
    print('First 2 scaled predictions')
    print(yhat_rescaled_all[0:2])
    print('Shape of predictions:', yhat_rescaled_all.shape)

    
    
    # 3) compare predictions with actuals / invert differencing:    
    if multivariate_flag == True:
        print('Invert Differencing of multivariate predictions...')    
        #invert differencing:  (adding value of previous timestep)
        predictions_all = invert_differencing(original_complete_dataset, yhat_rescaled_all, year_to_access)
        
        print('predictions preview:')
        print(predictions_all.head())
        
        # 4) get rmse for each timeseries
        rmse_per_ts = []
        for u in range(n_preds):
            rmse_single_ts = np.sqrt(mean_squared_error(original_complete_dataset.loc[year_to_access].iloc[:,u], predictions_all.iloc[:,u]))
            rmse_per_ts.append(rmse_single_ts)
            print('RMSE per TS {} for model: {}: {}'.format(u, model_name, rmse_per_ts[u]))
        
        #get average of all rmses
        total_rmse = np.mean(rmse_per_ts)
        print('Avg.RMSE for multivariate model: {}: {}'.format(model_name, total_rmse))
        
    else:
        #invert differencing:  (adding value of previous timestep)
        print('Invert Differencing of predictions...')        
        predictions_all = invert_differencing(original_complete_dataset, yhat_rescaled_all[:,0], year_to_access)
 
        print('predictions preview:')
        print(predictions_all.head())
        
        # 4) get rmse:
        rmse = np.sqrt(mean_squared_error(original_complete_dataset[year_to_access], predictions_all))
        print('RMSE for model: {}: {}'.format(model_name, rmse))
    
    
    #return RMSE results:
    rmse_results = []
    if multivariate_flag == True:
        rmse_results.append(total_rmse)
        rmse_results.append(rmse_per_ts)
   
    else:
        rmse_results.append(rmse)
        
    
    return predictions_all, rmse_results


# functions to store predictions on disk:

In [20]:
#safe predictions and history into df: (assuming models are trained with MSE: training and validation set)
def preds_into_df(preds_df, original_complete_dataset, multivariate_flag, year_to_store):    
    #add actuals to single df:
    if multivariate_flag == False:
        print('Add Actuals to df for single area...')
        #create dataframe to add column easily:
        preds_df = pd.DataFrame(preds_df)
        preds_df['actuals'] = original_complete_dataset[year_to_store] #store actual values to calculate RMSE quickly

    print('creation of preds_df done')
        
    return preds_df

def trainhistory_into_df(encoder_training_history, prediction_model_training_history, hist_col_labels):
    #create df for traning_history:
    hist_encoder_df = pd.DataFrame(encoder_training_history.history['loss'],columns=[hist_col_labels[0]])
    hist_encoder_df[hist_col_labels[1]] = encoder_training_history.history['val_loss']
    
    hist_prediction_df = pd.DataFrame(prediction_model_training_history.history['loss'],columns=[hist_col_labels[0]])
    hist_prediction_df[hist_col_labels[1]] = prediction_model_training_history.history['val_loss']
    
    print('creation of history_dfs done')
    
    return  hist_encoder_df, hist_prediction_df


#store predictions and training_history on disk:
def store_preds_and_trainhistory_on_disk(preds_df, hist_encoder_df, hist_prediction_df, preds_df_filename, hist_encoder_df_filename, hist_prediction_df_filename, Store_PATH):
    #get path where to store df:
    preds_df_final_path = os.path.join(Store_PATH, preds_df_filename)
    #store df:
    preds_df.to_csv(preds_df_final_path, header=True)
    
    #store history:
    hist_encoder_df_final_path = os.path.join(Store_PATH, hist_encoder_df_filename)
    hist_encoder_df.to_csv(hist_encoder_df_final_path, header=True)
    
    hist_prediction_df_final_path = os.path.join(Store_PATH, hist_prediction_df_filename)
    hist_prediction_df.to_csv(hist_prediction_df_final_path, header=True)
    
    print('Save df on disk done')


In [21]:
# save models to JSON -> check that weights_file name uses ".h5" format & model_file_name ".json"
def save_models_to_json(model_file_name, model_weights_file_name, Store_PATH, model):
    
    #create paths for model architecture & weights:
    model_final_path = Store_PATH + model_file_name    
    weights_final_path = Store_PATH + model_weights_file_name

    #store model & weights:
    model_as_json = model.to_json()
    with open(model_final_path, "w") as json_file:
        json_file.write(model_as_json)
    # serialize weights to HDF5
    model.save_weights(weights_final_path)
    print("Saved model to disk")


In [22]:
#store history + prediction_results in dicts diretly on disk with the help of other functions:
def store_results_of_dicts_on_disk(dict_to_access, validation_set_year, test_set_year, Df_Store_PATH, Model_Save_PATH, RMSE_Store_PATH, RMSE_df_name):
    
    #create & store dfs of Results:
    for key in dict_to_access:
        print('Store results of key: ', key)
                
        #store test_results:
        if 'multivariate' not in key:
            multivariate_flag = False
            
            #get prediction_df: 
            preds_df = preds_into_df(dict_to_access[key][0][2], dict_to_access[key][0][4], multivariate_flag, test_set_year)
            
            #create history dfs:
            hist_col_labels = ['loss (mse)','mae','val_loss (mse)','val_mae']
            hist_encoder_df, hist_prediction_df = trainhistory_into_df(dict_to_access[key][0][8], dict_to_access[key][0][0], hist_col_labels)

            #create validation_df:    
            valid_preds_df = dict_to_access[key][0][5] 
            valid_preds_df = pd.DataFrame(valid_preds_df)
            valid_preds_df['actuals'] = dict_to_access[key][0][4].loc[validation_set_year] #store actual values of original dataset                        
        

        else:
            multivariate_flag = True
            #get prediction_df: 
            preds_df = preds_into_df(dict_to_access[key][0][2], dict_to_access[key][0][4], multivariate_flag, test_set_year)    
            
            #create history dfs:
            hist_col_labels = ['loss (mse)','mae','val_loss (mse)','val_mae']
            hist_encoder_df, hist_prediction_df = trainhistory_into_df(dict_to_access[key][0][8], dict_to_access[key][0][0], hist_col_labels)
                  
            #create validation_df for multivariate case:  
            valid_preds_df = dict_to_access[key][0][5]           

   
        #create filenames:
        results_file = 'results_' + dict_to_access[key][0][3] + '.csv'
        history_encoder_file = 'history_autoencoder_' + dict_to_access[key][0][3] + '.csv'
        history_predict_file = 'history_predict_' + dict_to_access[key][0][3] + '.csv'
        valid_results_file = 'validation_results_' + dict_to_access[key][0][3] + '.csv'
              
        #store test results: 
        store_preds_and_trainhistory_on_disk(preds_df, hist_encoder_df, hist_prediction_df, results_file, history_encoder_file, history_predict_file, Df_Store_PATH)
        #store validation results:
        #get path where to store df:
        valid_df_final_path = os.path.join(Df_Store_PATH, valid_results_file)
        #store df:
        valid_preds_df.to_csv(valid_df_final_path, header=True)
  

        #store model architecture (architecture + weights):
        autoencoder_model_architecture_file = 'autoencoder_model_' + dict_to_access[key][0][3] + '.json'
        autoencoder_model_weights_file = 'autoencoder_model_' + dict_to_access[key][0][3] + '_weights.h5'
        predict_model_architecture_file = 'predict_model_' + dict_to_access[key][0][3] + '.json'
        predict_model_weights_file = 'predict_model_' + dict_to_access[key][0][3] + '_weights.h5'
        
        #call function to save models:   
        #save autoencoder model:
        save_models_to_json(autoencoder_model_architecture_file, autoencoder_model_weights_file, Model_Save_PATH, dict_to_access[key][0][9])
        #save predict model:
        save_models_to_json(predict_model_architecture_file, predict_model_weights_file, Model_Save_PATH, dict_to_access[key][0][1])

        
        #store RMSE results of models:
        #create dict to store RMSE results:
        dict_test_rmse = {}
        dict_valid_rmse = {}
        
        #add values to dict:
        #Note: for multivar_models only avg. RMSE is stored!!
        if 'multivar' in key:
            dict_valid_rmse[key] = []
            dict_test_rmse[key] = []
            #append avg. RMSE results of multivar model:
            dict_valid_rmse[key].append(dict_to_access[key][0][7][0])
            dict_test_rmse[key].append(dict_to_access[key][0][6][0])
        
        else:
            dict_test_rmse[key] = []
            dict_valid_rmse[key] = []
            dict_valid_rmse[key].append(dict_to_access[key][0][7])
            dict_test_rmse[key].append(dict_to_access[key][0][6])
        
        
    #create Df from rmse_dicts:
    rmse_valid_df = pd.DataFrame.from_dict(dict_valid_rmse, orient='index')
    rmse_test_df = pd.DataFrame.from_dict(dict_test_rmse, orient='index')

    #store results in one df:
    all_rmse_df = pd.concat([rmse_valid_df,rmse_test_df], axis=1, names=['RMSE_Validation','RMSE_Test'])
    all_rmse_df.columns = ['RMSE_Validation', 'RMSE_Test']

    #store df on disk:
    #Note: RMSE results are  stored in same path as prediction results
    #set name of df:
    RMSE_df_name_final = 'RMSE_Results_all_models_' + RMSE_df_name + '.csv'
    #store df:
    all_rmse_df.to_csv(RMSE_Store_PATH + RMSE_df_name_final, header=True)

    print('All dfs & models of dict stored!')
    
    


# get full autoencoder models:

In [23]:
''' Function to train selected autoencoder "automatically"... '''


def get_full_autoencoder_pred_model(ts_series, multivariate_flag, last_train_set_year, 
                                    validation_set_year, test_set_year, model_name, n_timesteps_T, 
                                    n_timesteps_F, n_preds,
                                    scale_range, standardizing_flag, stacked_encoder_flag, 
                                    n_hidden1_units_autoencod, n_hidden2_units_autoencod, 
                                    n_hidden1_units_pred, n_hidden2_units_pred, n_hidden3_units_pred, 
                                    n_batch_size_autoencoder, n_batch_size_pred_model, dropout_rate, 
                                    n_epochs, shuffle_flag, early_stopping_flag, uber_flag):       
    
    # 1) get data for model:
    if uber_flag == True:
        print('#Generate data for UBER Model...')
        #call function to genereate data for UBER Models:
        X_train_T, y_train_T, X_valid_T, y_valid_T, X_test_T, y_test_T, X_train_F_prev, X_valid_F_prev, X_train_F_fut, X_valid_F_fut, scaler_list = generate_UBER_data_autoencoder(ts_series, multivariate_flag, last_train_set_year, validation_set_year, test_set_year, n_timesteps_T, n_timesteps_F, n_preds, scale_range, standardizing_flag)
        
    else:
        print('#Generate data for regular Model...')
        #call regular function to get data for model:
        X_train, y_train, X_valid, y_valid, X_test, y_test, scaler_list = generate_data_autoencoder(ts_series, multivariate_flag, last_train_set_year, validation_set_year, test_set_year, n_timesteps_T, n_preds, scale_range, standardizing_flag)                                                 
  
  
    # 2) create autoencoder model:   
    if uber_flag == True:
        print('create UBER Autoencoder...')
        #get autoencoder:
        history_autoencoder, autoencoder_model = create_UBER_autoencoder_1H_with_RepeatVect(X_train_T, X_train_F_prev, X_train_F_fut, X_valid_T, X_valid_F_prev, X_valid_F_fut, n_timesteps_F, n_preds, n_hidden1_units_autoencod, n_hidden2_units_autoencod, dropout_rate, n_batch_size_autoencoder, n_epochs)

        #get the standalone encoder:
        #since the approach in "UBER-Paper" uses a prediciton net, based on dense layers, we need an encoder which outputs 2D outputs.. -> we take take last LSTM layer as output of encoder instead of RepeatVector..
        print('create standalone UBER encoder...')
        standalone_encoder = Model(inputs=autoencoder_model.layers[0].input, outputs=autoencoder_model.layers[1].output)
        #--> eventuell braucht man hier "inputs=autoencoder_model.inputs[0] oder so, da wir für standalone encoder nur x_train als input verwenden und nicht noch x_Train_F_prev..
    
    if stacked_encoder_flag == True and uber_flag == False:
        print('create stacked autoencoder 2 layer:')  
        #get autoencoder:
        history_autoencoder, autoencoder_model = create_stacked_autoencoder_2H(X_train, X_valid, n_timesteps_T, n_preds, n_hidden1_units_autoencod, n_hidden2_units_autoencod, dropout_rate, n_batch_size_autoencoder, n_epochs)
                 
        #get the standalone encoder:
        print('create standalone encoder:')  
        standalone_encoder = Model(inputs=autoencoder_model.inputs, outputs=autoencoder_model.layers[3].output)
        

    if stacked_encoder_flag == False and uber_flag == False:
        print('create autoencoder single layer:') 
        #get autoencoder:
        history_autoencoder, autoencoder_model = create_autoencoder_1H(X_train, X_valid, n_timesteps_T, n_preds, n_hidden1_units_autoencod, dropout_rate, n_batch_size_autoencoder, n_epochs)
        
        #get the standalone encoder:
        print('create standalone encoder:')  
        standalone_encoder = Model(inputs=autoencoder_model.inputs, outputs=autoencoder_model.layers[2].output)
    
    
    #prepare data for prediction model:
    #encode input:
    if uber_flag == True:
        print('encode data with UBER standalone encoder:')  
        X_train_encoded = standalone_encoder.predict(X_train_T)
        X_valid_encoded = standalone_encoder.predict(X_valid_T)
        X_test_encoded = standalone_encoder.predict(X_test_T)

        print('X_train_encoded shape: ', X_train_encoded.shape)
        print('X_valid_encoded shape: ', X_valid_encoded.shape)
        print('X_test_encoded shape: ', X_test_encoded.shape)
    
    else:
        print('encode data with standalone encoder:')  
        X_train_encoded = standalone_encoder.predict(X_train)
        X_valid_encoded = standalone_encoder.predict(X_valid)
        X_test_encoded = standalone_encoder.predict(X_test)

        print('X_train_encoded shape: ', X_train_encoded.shape)
        print('X_valid_encoded shape: ', X_valid_encoded.shape)
        print('X_test_encoded shape: ', X_test_encoded.shape)

    
    #create prediction model:
    if uber_flag == True:
        print('create UBER prediction_model:')   
        history_predict_model , prediction_model = create_UBER_prediction_model_3H(X_train_encoded, y_train_T, X_valid_encoded, y_valid_T, n_preds, n_hidden1_units_pred, n_hidden2_units_pred, n_hidden3_units_pred, dropout_rate, n_batch_size_pred_model, n_epochs, shuffle_flag)     

    else:
        print('create prediction_model:')                                           
        history_predict_model , prediction_model = create_prediction_model_2H(X_train_encoded, y_train, X_valid_encoded, y_valid, n_preds, n_hidden1_units_pred, n_hidden2_units_pred, dropout_rate, n_batch_size_pred_model, n_epochs, shuffle_flag, early_stopping_flag)
    
    #get prediction results:
    #results for valid_set:       
    validation_results, rmse_valid = get_rescaled_decoded_predictions(prediction_model, multivariate_flag, X_valid_encoded, validation_set_year, scaler_list, standardizing_flag, scale_range, n_preds, ts_series, 'validation_set')

    #results for test_set: 
    predictions_results, rmse_test = get_rescaled_decoded_predictions(prediction_model, multivariate_flag, X_test_encoded, test_set_year, scaler_list, standardizing_flag, scale_range, n_preds, ts_series, model_name)


    return history_predict_model, prediction_model, predictions_results, model_name, ts_series, validation_results, rmse_test, rmse_valid, history_autoencoder, autoencoder_model

            

# START Training of Models

## Vanilla Autoencoder

In [ ]:
'''Experiments vanilla Autoencoder with variation of hidden units & dimension of input data: '''


#create dict to store results, history, models...
autoencod_dict_results = {}


#area 237 training: 2009-2010, validation:2011 test:2012
ts_series_09_12_area237 = ts_10largest['2009':'2012'].iloc[:,0].copy()
ts_series_multivar = ts_10largest['2009':'2012'].iloc[:,:].copy()

#set parameters, valid for all experiments:
train_set_year = '2010'
validation_set_year = '2011'
test_set_year = '2012'

n_batch_size_autoencoder = 512
n_batch_size_pred_model = 512
dropout_rate = 0.3
n_epochs = 150

shuffle_flag = True    
early_stopping_flag = True
    

#tuples: (stacked_encoder, n_timesteps_T, n_timesteps_F, n_preds, multivariate_flag, uber_flag, scale_range, standardizing_flag, n_hidden1_units_autoencod,n_hidden2_units_autoencod,n_hidden1_units_pred,n_hidden2_units_pred,n_hidden3_units_pred)
experiment_settings = [(True, 168,24,1,False,False,(-1,1), False, 256,128,128,64,0),
                       (True, 168,24,1,False,False,(-1,1), True, 256,128,128,64,0),
                       (True, 168,24,10,True,False,(-1,1), False, 256,128,128,64,0),
                       (True, 168,24,10,True,False,(-1,1), True, 256,128,128,64,0),
                       (True, 168,24,1,False,False,(-1,1), False, 512,256,128,64,0),
                       (True, 168,24,10,True,False,(-1,1), False, 512,256,128,64,0),
                       (True, 24,24,1,False,False,(-1,1), False, 256,128,128,64,0),
                       (True, 24,24,10,True,False,(-1,1), False, 256,128,128,64,0),
                       (False, 168,24,1,False,False,(-1,1), False, 256,128,128,64,0),
                       (False, 168,24,10,True,False,(-1,1), False, 512,256,128,64,0)]


#get model & predictions:
for i in range(len(experiment_settings)):
      
    #get parameters:
    stacked_encoder_flag, n_timesteps_T, n_timesteps_F, n_preds, multivariate_flag, uber_flag, scale_range, standardizing_flag, n_hidden1_units_autoencod, n_hidden2_units_autoencod, n_hidden1_units_pred, n_hidden2_units_pred, n_hidden3_units_pred = experiment_settings[i]    
    
    #set model_name:
    if standardizing_flag == True:
        scaling = 'scaling_std'
    else:
        scaling = 'scaling_tanh'
 

    #set model_name:
    model_name = str(n_hidden1_units_autoencod) + '_' + str(n_hidden2_units_autoencod)+ '_1D_' + str(n_hidden2_units_autoencod) + '_predict_model_2H_' + str(n_hidden1_units_pred) + '_' + str(n_hidden2_units_pred) + '_batch512_drop03_' + scaling + '_W' + str(n_timesteps_T) 
    
    
    if stacked_encoder_flag == True:
        prefix = 'encoder_2H_'
    else:
        prefix = 'encoder_1H_'
        
    if multivariate_flag == True:
        prefix = 'multivariate_' + prefix
        postfix = '_10largest_areas'
        ts_series = ts_series_multivar
    else:
        postfix = '_area237'
        ts_series = ts_series_09_12_area237
   
    #final name:
    model_name = prefix + model_name + postfix + '__y2012'
    
    #add key to dict:
    autoencod_dict_results[model_name] = []
    
    print('######## START Experiment {}'.format(model_name))
        
    #call function to generate model and receive predictions:
    all_results_area_i = get_full_autoencoder_pred_model(ts_series, multivariate_flag, 
                                                         train_set_year, validation_set_year, test_set_year, 
                                                         model_name, n_timesteps_T, n_timesteps_F, n_preds, 
                                                         scale_range, standardizing_flag, stacked_encoder_flag, 
                                                         n_hidden1_units_autoencod, n_hidden2_units_autoencod, 
                                                         n_hidden1_units_pred, n_hidden2_units_pred, 
                                                         n_hidden3_units_pred, n_batch_size_autoencoder, 
                                                         n_batch_size_pred_model, dropout_rate, 
                                                         n_epochs, shuffle_flag, early_stopping_flag, 
                                                         uber_flag)       

    #append results in dict:
    autoencod_dict_results[model_name].append(all_results_area_i)

    





######## START Experiment encoder_2H_256_128_1D_128_predict_model_2H_128_64_batch512_drop03_scaling_tanh_W168_area237__y2012
#Generate data for regular Model...
complete valid_set shape:  (8928,)
complete test_set shape:  (8952,)
Data is scaled...
MinMax-Scaling used...
Train_data shape:  (17519, 1)
valid_data shape:  (8928, 1)
Test_data shape:  (8952, 1)
X_train shape for autoencoder:  (17351, 168, 1)
X_valid shape for autoencoder:  (8760, 168, 1)
X_test shape for autoencoder:  (8784, 168, 1)
y_train shape before model creation:  (17351, 1)
y_valid shape before model creation:  (8760, 1)
y_test shape before model creation:  (8784, 1)
create stacked autoencoder 2 layer:
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 17351 samples, validate on 8760 samples
Epoch 1/150
17351/17351 [============

17351/17351 [==============================] - 22s 1ms/step - loss: 0.0294 - val_loss: 0.0305
Epoch 138/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0245 - val_loss: 0.0284
Epoch 139/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0236 - val_loss: 0.0362
Epoch 140/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0225 - val_loss: 0.0290
Epoch 141/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0217 - val_loss: 0.0262
Epoch 142/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0241 - val_loss: 0.0272
Epoch 143/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0229 - val_loss: 0.0234
Epoch 144/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0303 - val_loss: 0.0366
Epoch 145/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.0273 - val_loss: 0.0285
Epoch 146/150
17351/17351 [===============

17351/17351 [==============================] - 16s 906us/step - loss: 0.0088 - mean_absolute_error: 0.0666 - val_loss: 0.0081 - val_mean_absolute_error: 0.0657
Epoch 86/150
17351/17351 [==============================] - 16s 900us/step - loss: 0.0088 - mean_absolute_error: 0.0662 - val_loss: 0.0080 - val_mean_absolute_error: 0.0648
Epoch 87/150
17351/17351 [==============================] - 16s 899us/step - loss: 0.0097 - mean_absolute_error: 0.0691 - val_loss: 0.0091 - val_mean_absolute_error: 0.0689
Epoch 88/150
17351/17351 [==============================] - 16s 907us/step - loss: 0.0090 - mean_absolute_error: 0.0671 - val_loss: 0.0080 - val_mean_absolute_error: 0.0647
Epoch 89/150
17351/17351 [==============================] - 16s 899us/step - loss: 0.0087 - mean_absolute_error: 0.0661 - val_loss: 0.0082 - val_mean_absolute_error: 0.0654
Epoch 90/150
17351/17351 [==============================] - 16s 900us/step - loss: 0.0087 - mean_absolute_error: 0.0659 - val_loss: 0.0079 - val_mea

17351/17351 [==============================] - 22s 1ms/step - loss: 0.9908 - val_loss: 1.1738
Epoch 33/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9832 - val_loss: 1.1684
Epoch 34/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9756 - val_loss: 1.1615
Epoch 35/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9793 - val_loss: 1.1744
Epoch 36/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9798 - val_loss: 1.1612
Epoch 37/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9686 - val_loss: 1.1474
Epoch 38/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9632 - val_loss: 1.1555
Epoch 39/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9535 - val_loss: 1.4565
Epoch 40/150
17351/17351 [==============================] - 22s 1ms/step - loss: 0.9856 - val_loss: 1.1421
Epoch 41/150
17351/17351 [========================

17351/17351 [==============================] - 15s 888us/step - loss: 0.3486 - mean_absolute_error: 0.4280 - val_loss: 0.3220 - val_mean_absolute_error: 0.4222
Epoch 21/150
17351/17351 [==============================] - 16s 898us/step - loss: 0.3596 - mean_absolute_error: 0.4335 - val_loss: 0.3388 - val_mean_absolute_error: 0.4305
Epoch 22/150
17351/17351 [==============================] - 16s 895us/step - loss: 0.3619 - mean_absolute_error: 0.4346 - val_loss: 0.3467 - val_mean_absolute_error: 0.4297
Epoch 23/150
17351/17351 [==============================] - 16s 895us/step - loss: 0.3874 - mean_absolute_error: 0.4476 - val_loss: 0.3370 - val_mean_absolute_error: 0.4371
Epoch 24/150
17351/17351 [==============================] - 16s 910us/step - loss: 0.3273 - mean_absolute_error: 0.4190 - val_loss: 0.3305 - val_mean_absolute_error: 0.4193
Epoch 25/150
17351/17351 [==============================] - 16s 911us/step - loss: 0.3212 - mean_absolute_error: 0.4116 - val_loss: 0.2964 - val_mea

...

17351/17351 [==============================] - 16s 908us/step - loss: 0.3147 - mean_absolute_error: 0.3911 - val_loss: 0.3241 - val_mean_absolute_error: 0.3980
Epoch 105/150
17351/17351 [==============================] - 16s 895us/step - loss: 0.3136 - mean_absolute_error: 0.3897 - val_loss: 0.3235 - val_mean_absolute_error: 0.3984
Epoch 106/150
17351/17351 [==============================] - 16s 908us/step - loss: 0.3214 - mean_absolute_error: 0.3953 - val_loss: 0.3309 - val_mean_absolute_error: 0.4018
Epoch 107/150
17351/17351 [==============================] - 16s 895us/step - loss: 0.3163 - mean_absolute_error: 0.3927 - val_loss: 0.3308 - val_mean_absolute_error: 0.4020
Epoch 108/150
17351/17351 [==============================] - 16s 897us/step - loss: 0.3161 - mean_absolute_error: 0.3917 - val_loss: 0.3304 - val_mean_absolute_error: 0.4014
Epoch 109/150
17351/17351 [==============================] - 16s 897us/step - loss: 0.3252 - mean_absolute_error: 0.3982 - val_loss: 0.3353 - va

17351/17351 [==============================] - 43s 2ms/step - loss: 0.0422 - val_loss: 0.0503
Epoch 30/150
17351/17351 [==============================] - 43s 2ms/step - loss: 0.0421 - val_loss: 0.0502
Epoch 31/150
17351/17351 [==============================] - 43s 3ms/step - loss: 0.0421 - val_loss: 0.0501
Epoch 32/150
17351/17351 [==============================] - 43s 2ms/step - loss: 0.0419 - val_loss: 0.0497
Epoch 33/150
17351/17351 [==============================] - 43s 3ms/step - loss: 0.0418 - val_loss: 0.0504
Epoch 34/150
17351/17351 [==============================] - 43s 2ms/step - loss: 0.0422 - val_loss: 0.0504
Epoch 35/150
17351/17351 [==============================] - 43s 3ms/step - loss: 0.0422 - val_loss: 0.0504
Epoch 36/150
17351/17351 [==============================] - 43s 2ms/step - loss: 0.0422 - val_loss: 0.0503
Epoch 37/150
17351/17351 [==============================] - 43s 3ms/step - loss: 0.0422 - val_loss: 0.0503
Epoch 38/150
17351/17351 [========================

17351/17351 [==============================] - 17s 952us/step - loss: 0.0354 - mean_absolute_error: 0.1382 - val_loss: 0.0329 - val_mean_absolute_error: 0.1375
Epoch 19/150
17351/17351 [==============================] - 17s 964us/step - loss: 0.0339 - mean_absolute_error: 0.1359 - val_loss: 0.0388 - val_mean_absolute_error: 0.1531
Epoch 20/150
17351/17351 [==============================] - 17s 953us/step - loss: 0.0357 - mean_absolute_error: 0.1389 - val_loss: 0.0346 - val_mean_absolute_error: 0.1305
Epoch 21/150
17351/17351 [==============================] - 17s 951us/step - loss: 0.0340 - mean_absolute_error: 0.1355 - val_loss: 0.0345 - val_mean_absolute_error: 0.1389
Epoch 22/150
17351/17351 [==============================] - 17s 953us/step - loss: 0.0327 - mean_absolute_error: 0.1336 - val_loss: 0.0293 - val_mean_absolute_error: 0.1174
Epoch 23/150
17351/17351 [==============================] - 17s 965us/step - loss: 0.0334 - mean_absolute_error: 0.1340 - val_loss: 0.0303 - val_mea

17351/17351 [==============================] - 44s 3ms/step - loss: 0.0374 - val_loss: 0.0426
Epoch 39/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0374 - val_loss: 0.0425
Epoch 40/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0373 - val_loss: 0.0425
Epoch 41/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0373 - val_loss: 0.0423
Epoch 42/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0370 - val_loss: 0.0422
Epoch 43/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0393 - val_loss: 0.0432
Epoch 44/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0376 - val_loss: 0.0427
Epoch 45/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0375 - val_loss: 0.0426
Epoch 46/150
17351/17351 [==============================] - 44s 3ms/step - loss: 0.0374 - val_loss: 0.0426
Epoch 47/150
17351/17351 [========================

Epoch 24/150
17351/17351 [==============================] - 17s 959us/step - loss: 0.0148 - mean_absolute_error: 0.0861 - val_loss: 0.0151 - val_mean_absolute_error: 0.0870
Epoch 25/150
17351/17351 [==============================] - 17s 967us/step - loss: 0.0145 - mean_absolute_error: 0.0855 - val_loss: 0.0149 - val_mean_absolute_error: 0.0863
Epoch 26/150
17351/17351 [==============================] - 17s 967us/step - loss: 0.0154 - mean_absolute_error: 0.0877 - val_loss: 0.0177 - val_mean_absolute_error: 0.0943
Epoch 27/150
17351/17351 [==============================] - 16s 951us/step - loss: 0.0166 - mean_absolute_error: 0.0915 - val_loss: 0.0159 - val_mean_absolute_error: 0.0893
Epoch 28/150
17351/17351 [==============================] - 17s 954us/step - loss: 0.0152 - mean_absolute_error: 0.0875 - val_loss: 0.0153 - val_mean_absolute_error: 0.0876
Epoch 29/150
17351/17351 [==============================] - 17s 956us/step - loss: 0.0148 - mean_absolute_error: 0.0865 - val_loss: 0.0

## Uber Autoencoder

In [52]:
'Experiments UBER Autoencoder'

'''
Different experiments are run with the UBER Autoencoder architecture: 

   - variation of hidden units
   - variation of input dimensions: matrix ('single') vs tensor ('multivariate')

'''

#create dict to store results, history, models...
uber_autoencod_dict_results = {}


#set parameters, valid for all experiments:
start_train_set_year = '2009'
last_train_set_year = '2010'
validation_set_year = '2011'
test_set_year = '2012'

#area 237 training: 2009-2010, validation:2011 test:2012
ts_series_multivar = ts_10largest[start_train_set_year:'2013'].iloc[:,:].copy() #Note: we have to set end of data set to '2013' -> this way the future values of last entry of '2012' can be extracted of 2013
ts_series_single = ts_10largest[start_train_set_year:'2013'].iloc[:,0].copy()

#set parameters for model:
n_batch_size_autoencoder = 512
n_batch_size_pred_model = 512
dropout_rate = 0.3
n_epochs = 150

shuffle_flag = True

stacked_encoder_flag = False

early_stopping_flag = False

scale_range = (-1,1) 
standardizing_flag = False


#tuples: (n_timesteps_T, n_timesteps_F, n_preds, multivariate_flag, uber_flag, n_hidden1_units_autoencod,n_hidden2_units_autoencod,n_hidden1_units_pred,n_hidden2_units_pred,n_hidden3_units_pred)
experiment_settings = [(168,24,1,False,True,256,128,128,64,16),
                       (168,24,1,False,True,256,128,256,128,16),
                       (168,24,1,False,True,512,128,128,64,16),
                       (168,24,1,False,True,512,256,512,128,16),
                       (168,24,10,True,True,256,128,128,64,16),
                       (168,24,10,True,True,256,128,256,128,16),
                       (168,24,10,True,True,512,128,128,64,16),
                       (168,24,10,True,True,512,256,512,128,16)]

                                                             
#get model & predictions:
for i in range(len(experiment_settings)):
      
    #get parameters:
    n_timesteps_T, n_timesteps_F, n_preds, multivariate_flag, uber_flag, n_hidden1_units_autoencod,n_hidden2_units_autoencod,n_hidden1_units_pred, n_hidden2_units_pred, n_hidden3_units_pred = experiment_settings[i]
    
    
    #set model_name:    
    model_name = str(n_hidden1_units_autoencod) + '_1D_' + str(n_hidden2_units_autoencod) + '_predict_model_3H_' + str(n_hidden1_units_pred) + '_' + str(n_hidden2_units_pred) + '_' + str(n_hidden3_units_pred) + '_batch512_drop03_scaling_tanh_W' + str(n_timesteps_T)

    prefix = 'UBER_auto_encoder_'
        
    if multivariate_flag == True:
        postfix = '_multivariate_10largest_areas__y2012'       
    else:
        postfix = '_area237__y2012'
        
    if uber_flag == True:
        uber_fix = '_F{}'.format(n_timesteps_F)
    else:
        uber_fix = ''
        
    
    #final name:
    model_name = prefix + model_name  + uber_fix + postfix
    
    print('### START Experiment with ', model_name)
    
    
    #set dataset:
    if multivariate_flag == True:
        ts_series = ts_series_multivar
    else: 
        ts_series = ts_series_single 
        
    #add key to dict:
    uber_autoencod_dict_results[model_name] = []
        
    #call function to generate model and receive predictions:
    all_results_model_i = get_full_autoencoder_pred_model(ts_series, multivariate_flag, last_train_set_year, 
                                                          validation_set_year, test_set_year, model_name, 
                                                          n_timesteps_T, n_timesteps_F, n_preds, scale_range, 
                                                          standardizing_flag, stacked_encoder_flag, 
                                                          n_hidden1_units_autoencod, n_hidden2_units_autoencod, 
                                                          n_hidden1_units_pred, n_hidden2_units_pred, 
                                                          n_hidden3_units_pred, n_batch_size_autoencoder, 
                                                          n_batch_size_pred_model, dropout_rate, n_epochs, 
                                                          shuffle_flag, early_stopping_flag, uber_flag)      

    #append results in dict:
    uber_autoencod_dict_results[model_name].append(all_results_model_i)

    


### START Experiment with  UBER_auto_encoder_256_1D_128_predict_model_3H_128_64_16_batch512_drop03_scaling_tanh_W168_F24_area237__y2012
#Generate data for UBER Model...
data for area237 is prepared...
Data is scaled...
MinMax-Scaling used...
shape of X_train single area:  (17351, 168, 1)
shape of X_valid single area:  (8760, 168, 1)
shape of X_test single area:  (8784, 168, 1)
shape of y_train single area:  (17351, 1)
shape of y_valid single area:  (8760, 1)
shape of y_test single area:  (8784, 1)
shape of prev_lags_F_train single area:  (17351, 24, 1)
shape of fut_F_values_train single area:  (17351, 24, 1)
final shape of X_train:  (17351, 168, 1)
create UBER Autoencoder...
Train on 17351 samples, validate on 8760 samples
Epoch 1/150
17351/17351 [==============================] - 11s 640us/step - loss: 0.0415 - val_loss: 0.0439
Epoch 2/150
17351/17351 [==============================] - 9s 498us/step - loss: 0.0359 - val_loss: 0.0388
Epoch 3/150
17351/17351 [===========================

17351/17351 [==============================] - 9s 502us/step - loss: 0.0063 - val_loss: 0.0084
Epoch 146/150
17351/17351 [==============================] - 9s 497us/step - loss: 0.0061 - val_loss: 0.0081
Epoch 147/150
17351/17351 [==============================] - 8s 490us/step - loss: 0.0061 - val_loss: 0.0081
Epoch 148/150
17351/17351 [==============================] - 9s 504us/step - loss: 0.0062 - val_loss: 0.0084
Epoch 149/150
17351/17351 [==============================] - 9s 495us/step - loss: 0.0061 - val_loss: 0.0083
Epoch 150/150
17351/17351 [==============================] - 9s 499us/step - loss: 0.0059 - val_loss: 0.0084
create standalone UBER encoder...
encode data with UBER standalone encoder:
X_train_encoded shape:  (17351, 256)
X_valid_encoded shape:  (8760, 256)
X_test_encoded shape:  (8784, 256)
create UBER prediction_model:
Train on 17351 samples, validate on 8760 samples
Epoch 1/150
17351/17351 [==============================] - 2s 109us/step - loss: 0.0969 - mean_ab

Epoch 44/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0107 - mean_absolute_error: 0.0770 - val_loss: 0.0113 - val_mean_absolute_error: 0.0780
Epoch 45/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0106 - mean_absolute_error: 0.0763 - val_loss: 0.0114 - val_mean_absolute_error: 0.0783
Epoch 46/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0107 - mean_absolute_error: 0.0765 - val_loss: 0.0117 - val_mean_absolute_error: 0.0789
Epoch 47/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0106 - mean_absolute_error: 0.0764 - val_loss: 0.0114 - val_mean_absolute_error: 0.0788
Epoch 48/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0107 - mean_absolute_error: 0.0764 - val_loss: 0.0112 - val_mean_absolute_error: 0.0772
Epoch 49/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0104 - mean_absolute_error: 0.0754 - val_loss: 0.0115 - val_me

Epoch 92/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0097 - mean_absolute_error: 0.0732 - val_loss: 0.0108 - val_mean_absolute_error: 0.0752
Epoch 93/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0098 - mean_absolute_error: 0.0727 - val_loss: 0.0105 - val_mean_absolute_error: 0.0741
Epoch 94/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0099 - mean_absolute_error: 0.0729 - val_loss: 0.0109 - val_mean_absolute_error: 0.0757
Epoch 95/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0097 - mean_absolute_error: 0.0725 - val_loss: 0.0107 - val_mean_absolute_error: 0.0749
Epoch 96/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0096 - mean_absolute_error: 0.0720 - val_loss: 0.0109 - val_mean_absolute_error: 0.0764
Epoch 97/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0095 - mean_absolute_error: 0.0720 - val_loss: 0.0106 - val_me

Epoch 140/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0094 - mean_absolute_error: 0.0713 - val_loss: 0.0103 - val_mean_absolute_error: 0.0735
Epoch 141/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0093 - mean_absolute_error: 0.0704 - val_loss: 0.0104 - val_mean_absolute_error: 0.0738
Epoch 142/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0095 - mean_absolute_error: 0.0714 - val_loss: 0.0105 - val_mean_absolute_error: 0.0748
Epoch 143/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0093 - mean_absolute_error: 0.0710 - val_loss: 0.0107 - val_mean_absolute_error: 0.0754
Epoch 144/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0095 - mean_absolute_error: 0.0715 - val_loss: 0.0109 - val_mean_absolute_error: 0.0759
Epoch 145/150
17351/17351 [==============================] - 0s 12us/step - loss: 0.0094 - mean_absolute_error: 0.0710 - val_loss: 0.0104 - 

...

17351/17351 [==============================] - 14s 802us/step - loss: 0.0063 - val_loss: 0.0097
Epoch 114/150
17351/17351 [==============================] - 14s 799us/step - loss: 0.0062 - val_loss: 0.0098
Epoch 115/150
17351/17351 [==============================] - 14s 795us/step - loss: 0.0062 - val_loss: 0.0097
Epoch 116/150
17351/17351 [==============================] - 14s 800us/step - loss: 0.0063 - val_loss: 0.0097
Epoch 117/150
17351/17351 [==============================] - 14s 796us/step - loss: 0.0063 - val_loss: 0.0100
Epoch 118/150
17351/17351 [==============================] - 14s 800us/step - loss: 0.0062 - val_loss: 0.0099
Epoch 119/150
17351/17351 [==============================] - 14s 802us/step - loss: 0.0061 - val_loss: 0.0099
Epoch 120/150
17351/17351 [==============================] - 14s 801us/step - loss: 0.0061 - val_loss: 0.0103
Epoch 121/150
17351/17351 [==============================] - 14s 799us/step - loss: 0.0061 - val_loss: 0.0098
Epoch 122/150
17351/1735

17351/17351 [==============================] - 0s 14us/step - loss: 0.0150 - mean_absolute_error: 0.0898 - val_loss: 0.0138 - val_mean_absolute_error: 0.0838
Epoch 24/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0149 - mean_absolute_error: 0.0896 - val_loss: 0.0138 - val_mean_absolute_error: 0.0836
Epoch 25/150
17351/17351 [==============================] - 0s 15us/step - loss: 0.0147 - mean_absolute_error: 0.0889 - val_loss: 0.0137 - val_mean_absolute_error: 0.0835
Epoch 26/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0146 - mean_absolute_error: 0.0883 - val_loss: 0.0136 - val_mean_absolute_error: 0.0833
Epoch 27/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0145 - mean_absolute_error: 0.0883 - val_loss: 0.0139 - val_mean_absolute_error: 0.0840
Epoch 28/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0144 - mean_absolute_error: 0.0879 - val_loss: 0.0138 - val_mean_absolute_e

Epoch 71/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0131 - mean_absolute_error: 0.0831 - val_loss: 0.0135 - val_mean_absolute_error: 0.0831
Epoch 72/150
17351/17351 [==============================] - 0s 15us/step - loss: 0.0132 - mean_absolute_error: 0.0832 - val_loss: 0.0134 - val_mean_absolute_error: 0.0827
Epoch 73/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0131 - mean_absolute_error: 0.0830 - val_loss: 0.0133 - val_mean_absolute_error: 0.0822
Epoch 74/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0131 - mean_absolute_error: 0.0831 - val_loss: 0.0133 - val_mean_absolute_error: 0.0825
Epoch 75/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0131 - mean_absolute_error: 0.0831 - val_loss: 0.0133 - val_mean_absolute_error: 0.0826
Epoch 76/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0132 - mean_absolute_error: 0.0832 - val_loss: 0.0134 - val_me

Epoch 119/150
17351/17351 [==============================] - 0s 15us/step - loss: 0.0129 - mean_absolute_error: 0.0825 - val_loss: 0.0132 - val_mean_absolute_error: 0.0822
Epoch 120/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0128 - mean_absolute_error: 0.0822 - val_loss: 0.0129 - val_mean_absolute_error: 0.0813
Epoch 121/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0128 - mean_absolute_error: 0.0825 - val_loss: 0.0130 - val_mean_absolute_error: 0.0814
Epoch 122/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0128 - mean_absolute_error: 0.0823 - val_loss: 0.0134 - val_mean_absolute_error: 0.0830
Epoch 123/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0128 - mean_absolute_error: 0.0823 - val_loss: 0.0131 - val_mean_absolute_error: 0.0817
Epoch 124/150
17351/17351 [==============================] - 0s 14us/step - loss: 0.0128 - mean_absolute_error: 0.0823 - val_loss: 0.0129 - 

predictions preview:
                            237         161         230           79  \
date                                                                   
2012-01-01 00:00:00  418.573616  346.502472  115.861198  1325.129628   
2012-01-01 01:00:00  357.448296  248.442932   -1.217415   624.956871   
2012-01-01 02:00:00  428.143951  155.423538  -17.010376   590.210312   
2012-01-01 03:00:00  284.621765  169.799332  -23.464325   510.283081   
2012-01-01 04:00:00  135.226704  161.031380   80.276512   391.604805   

                            236         162         170         234  \
date                                                                  
2012-01-01 00:00:00  483.848183  337.529495  558.097908  667.279778   
2012-01-01 01:00:00  312.328041  205.128548  417.310188  428.767601   
2012-01-01 02:00:00  388.564461  261.020020  463.310867  232.156052   
2012-01-01 03:00:00  334.962860  286.120903  451.496971  179.735504   
2012-01-01 04:00:00  179.687802  194.493263  425

In [54]:
#store results of UBER Autoencoder experiments on disk:


#prepare input_parameters:
validation_set_year = '2011'
test_set_year = '2012'

#store results on disk:
df_Store_PATH = '/media/...'
Model_Store_PATH = '/media/...'
RMSE_Store_PATH = '/media/...'
RMSE_df_name = 'UBER_autoencoder_various_hidden_units'


#call function to store results:
store_results_of_dicts_on_disk(uber_autoencod_dict_results, validation_set_year, test_set_year, df_Store_PATH, Model_Store_PATH, RMSE_Store_PATH, RMSE_df_name)



Store results of key:  UBER_auto_encoder_256_1D_128_predict_model_3H_128_64_16_batch512_drop03_scaling_tanh_W168_F24_area237__y2012
Add Actuals to df for single area...
creation of preds_df done
creation of history_dfs done
Save df on disk done
Saved model to disk
Saved model to disk
Store results of key:  UBER_auto_encoder_256_1D_128_predict_model_3H_256_128_16_batch512_drop03_scaling_tanh_W168_F24_area237__y2012
Add Actuals to df for single area...
creation of preds_df done
creation of history_dfs done
Save df on disk done
Saved model to disk
Saved model to disk
Store results of key:  UBER_auto_encoder_512_1D_128_predict_model_3H_128_64_16_batch512_drop03_scaling_tanh_W168_F24_area237__y2012
Add Actuals to df for single area...
creation of preds_df done
creation of history_dfs done
Save df on disk done
Saved model to disk
Saved model to disk
Store results of key:  UBER_auto_encoder_512_1D_256_predict_model_3H_512_128_16_batch512_drop03_scaling_tanh_W168_F24_area237__y2012
Add Actuals